In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('aapl.us.csv')

# discard data before year 2005
df = df[(df['Date'] < '2005-01-01')]
df

,Date,Open,High,Low,Close,Volume,OpenInt
0,1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0
1,1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0
2,1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0
3,1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0
4,1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0
...,...,...,...,...,...,...,...
5120,2004-12-27,4.14690,4.17210,4.02630,4.04420,156165143,0
5121,2004-12-28,4.05450,4.11340,3.97230,4.10950,170662716,0
5122,2004-12-29,4.09180,4.16070,4.07120,4.12610,125388393,0
5123,2004-12-30,4.14690,4.16460,4.11210,4.14910,96309465,0


In [21]:
# daily: 5 day to predict next 1 day, X: close price and volume, y: close price

df_day_5_1 = pd.DataFrame()

# volume
df_day_5_1['Volume (t-5)'] = df['Volume']
df_day_5_1['Volume (t-4)'] = df['Volume'].shift(-1)
df_day_5_1['Volume (t-3)'] = df['Volume'].shift(-2)
df_day_5_1['Volume (t-2)'] = df['Volume'].shift(-3)
df_day_5_1['Volume (t-1)'] = df['Volume'].shift(-4)

# close price
df_day_5_1['Close (t-5)'] = df['Close']
df_day_5_1['Close (t-4)'] = df['Close'].shift(-1)
df_day_5_1['Close (t-3)'] = df['Close'].shift(-2)
df_day_5_1['Close (t-2)'] = df['Close'].shift(-3)
df_day_5_1['Close (t-1)'] = df['Close'].shift(-4)

# y-variable (close price)
df_day_5_1['Close (t)'] = df['Close'].shift(-5)
df_day_5_1 = df_day_5_1.dropna()

# y-variable (movement)
df_day_5_1['movement'] = np.where(df_day_5_1["Close (t)"] > df_day_5_1['Close (t-1)'], 1, 0)
df_day_5_1.to_csv('df_day_5_1.csv')

In [24]:
# split data to train and test

def split(filename, n):
    df = pd.read_csv(filename)
    train_size = round(len(df)*0.7)
    y_movement = df['movement']
    y_t = df['Close (t)']

    if n == 1:
        t_n = ['t-1']
    else:
        t_n = ['Volume (t-5)', 'Volume (t-4)', 'Volume (t-3)', 'Volume (t-2)', 'Volume (t-1)',\
                   'Close (t-5)', 'Close (t-4)', 'Close (t-3)', 'Close (t-2)', 'Close (t-1)']
    X = df[t_n]

    # for movement
    X_train, X_test, y_train_movement, y_test_movement = X[:train_size], X[train_size:], y_movement[:train_size], y_movement[train_size:]

    # for t
    X_train, X_test, y_train_t, y_test_t = X[:train_size], X[train_size:], y_t[:train_size], y_t[train_size:]
    train_df = pd.concat([X_train,y_train_t,y_train_movement],axis = 1)
    test_df = pd.concat([X_test,y_test_t, y_test_movement],axis = 1)
    return train_df, test_df

In [25]:
train, test = split('df_day_5_1.csv', 5)
train.to_csv('train_df_day_5_1.csv', index = False)
test.to_csv('test_df_day_5_1.csv', index = False)